In [46]:
#!pip install rdflib
#!pip install pydot2
#!pip install pydotplus

In [47]:
import rdflib
from rdflib.namespace import RDFS
from rdflib import URIRef, BNode, Literal
import re
from collections import defaultdict
import numpy as np
import pandas as pd
import pydot as pydot
import networkx as nx
from networkx.drawing.nx_pydot import write_dot
import matplotlib.pyplot as plt

In [48]:
def ns(url):
  url = url.replace("http://buildsys.org/ontologies/brick#", "")
  url = url.replace("http://buildsys.org/ontologies/brickFrame#", "")
  return url

BRICKF = rdflib.Namespace('http://buildsys.org/ontologies/brickFrame#')
BRICK  = rdflib.Namespace('http://buildsys.org/ontologies/brick#')
TAG    = rdflib.Namespace('http://buildsys.org/ontologies/brickTag#')
TAGSET = rdflib.Namespace('http://buildsys.org/ontologies/brickTagSet#')

g = rdflib.Graph()
g.bind('bf', BRICKF)
g.bind('tag', TAG)
g.bind('ts', TAGSET)
g.bind(':', BRICK)
result = g.parse('BrickFrame.ttl', format='n3')
print(len(g))

123


### Load Tag and TagSets from Definition

In [49]:
dfTags=pd.read_excel('Schema Engineering.xlsx',"Tags")
schemaTags=set(pd.unique(dfTags.Tag.dropna().ravel()))
len(schemaTags)
dfTags.head()

,Dimension,Tag,Definition,Related Terminologies,hasSynonym,Author,Reviewed,Review Count,Comments
0,Equipment,HVAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Equipment>Energy>Power,Load,NaN,Eletrical load,NaN,Jason,NaN,NaN,NaN
2,Equipment>Energy>Power,Bus,"A metallic strip or bar (typically copper, bra...",NaN,NaN,Jason,NaN,NaN,NaN
3,Equipment>Energy>Power,Battery,NaN,NaN,NaN,Jason,NaN,NaN,NaN
4,Equipment>Energy>Power,Solar,NaN,Solar panel,NaN,Jason,NaN,NaN,NaN


In [50]:
dfTagSets=pd.read_excel('Schema Engineering.xlsx',"TagSets")
schemaTagSets=set()
for ts in pd.unique(dfTagSets.TagSet.dropna().ravel()):
  schemaTagSets.add(ts.replace(' ','_'))
for ts in pd.unique(dfTagSets.hasSynonym.dropna().ravel()):
  for ts2 in ts.split(","):
   schemaTagSets.add(ts2.replace(' ','_'))
len(schemaTagSets)
dfTagSets.head()

,Dimension,TagSet,hasSynonym,Definition,hasUnit,hasAbbreviation,can be named? (can be instantiated?),can be unnamed? (cannot be instantiated?),Author,Reviewed,Review Count,Discussion
0,Equipment>ExhaustSystem,Exhaust Fan,NaN,NaN,NaN,NaN,NaN,NaN,Jason,NaN,NaN,NaN
1,Equipment>HVAC,Chilled Water System,NaN,NaN,NaN,NaN,O,X,Jason,NaN,NaN,NaN
2,Equipment>HVAC,Fire Control Panel,NaN,NaN,NaN,FCP,NaN,NaN,Bharath,NaN,NaN,NaN
3,Equipment>HVAC,Space Heater,NaN,NaN,NaN,NaN,O,X,Jason,NaN,NaN,NaN
4,Equipment>HVAC,Air Handler Unit,NaN,A device used to regulate and circulate air as...,NaN,AHU,O,X,Joern,NaN,NaN,NaN


In [51]:
schemaUsedTags=set()
schemaTagSetTags={}
for ts in schemaTagSets:
  schemaUsedTags.update(ts.split('_'))
  schemaTagSetTags[ts]=set(ts.split('_'))
schemaMissingTags=schemaUsedTags - schemaTags
print("Missing Tags:" + str(len(schemaMissingTags)))
print(schemaMissingTags)

Missing Tags:1
{''}


In [52]:
def IndivName(name):
  return re.sub(r'\s','_',re.sub(r'[^\d\w\s]', '', name))

### Write Tags

write header

In [53]:
nsTag   = ":"
fo = open('BrickTag.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix : <http://buildsys.org/ontologies/BrickTag#> .\n\n")
fo.write("<http://buildsys.org/ontologies/BrickTag>  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickFrame> ;\n")
fo.write('\trdfs:comment "Domain Tag Definition"@en .\n\n')

44

Write Tag Hierachy

In [54]:
brickTags={}
for hir in pd.unique(dfTags.Dimension.dropna().ravel()):
  tags=hir.split('>')
  atags=""
  for i in range(len(tags)):
    tag=tags[i]
    otags=atags
    atags=(atags+"_"+tag).strip("_")
    if atags not in brickTags:
      indivLocName=nsTag+IndivName(atags)
      if i>0:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+brickTags[otags]+";")
      else:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   bf:Tag;")
      fo.write('\n\t\t\t bf:isHierarchical  "";')
      fo.write('\n\t\t\t rdfs:label "'+tag+'"@en .\n')
      brickTags[atags]=indivLocName;
      parent=tag;

Add tag leaves

In [55]:
# create location individuals
for idx in dfTags.index:
  #parent=brickTags[str(dfTags.loc[idx, "Dimension"]).split('>')[-1]]
  parent=brickTags[str(dfTags.loc[idx, "Dimension"]).replace('>','_')]
  indivLocName=nsTag + IndivName(str(dfTags.loc[idx, "Tag"]))
  fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+parent+";")
  fo.write('\n\t\t\t rdfs:label "'+str(dfTags.loc[idx, "Tag"])+'"@en .\n')
  brickTags[tag]=indivLocName;
  #print(parent,indivLocName)

In [56]:
fo.close()

### Write TagSets

write headers

In [57]:
nsTag= "tag:" # "ts:"
nsTagSet= ":" # "ts:"
fo = open('Brick.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix tag: <http://buildsys.org/ontologies/BrickTag#> .\n")
fo.write("@prefix :    <http://buildsys.org/ontologies/Brick#> .\n\n")
fo.write("<http://buildsys.org/ontologies/Brick>  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickFrame> ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickTag> ;\n")
fo.write('\trdfs:comment "Domain TagSet Definition"@en .\n\n')

47

Write TagSet Hierachy

In [58]:
brickTagSets={}
for hir in pd.unique(dfTagSets.Dimension.dropna().ravel()):
  tags=hir.split('>')
  atags=""
  for i in range(len(tags)):
    tag=tags[i]
    otags=atags
    atags=(atags+"_"+tag).strip('_')
    if atags not in brickTagSets:
      indivLocName=nsTagSet + IndivName(atags)
      if i>0:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+brickTagSets[otags]+";")
      else:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   bf:TagSet;")
      fo.write('\n\t\t\t bf:isHierarchical  "";')
      fo.write('\n\t\t\t rdfs:label "'+tag+'"@en .\n')
      brickTagSets[atags]=indivLocName;

Add TagSets leaves

In [59]:
# create location individuals
for idx in dfTagSets.index:
  tagsets=set([str(dfTagSets.loc[idx, "TagSet"])]) | set(str(dfTagSets.loc[idx, "hasSynonym"]).split(","))
  ots=None
  for tagset in tagsets:
    if tagset!="nan":
        #tagset=str(dfTagSets.loc[idx, "TagSet"])
      #parent= brickTagSets[str(dfTagSets.loc[idx, "Dimension"]).split('>')[-1]]
      parent= brickTagSets[str(dfTagSets.loc[idx, "Dimension"]).replace('>',"_")]
      indivLocName=nsTagSet + IndivName(tagset)
      fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+parent+";")
      for tag in tagset.split():
         #fo.write('\n\t\t\t bf:hasTag  tag:'+tag+';')
         fo.write('\n\t\t\t rdfs:subClassOf [ a owl:Restriction ; owl:onProperty :hasTag ; owl:someValuesFrom tag:'+tag+' . ];')
      if ots:
        fo.write('\n\t\t\t owl:equivalentClass '+ots+' ;')
      fo.write('\n\t\t\t rdfs:label "'+str(dfTagSets.loc[idx, "TagSet"])+'"@en .\n')
      brickTagSets[tagset]=indivLocName;
      ots=indivLocName;
      #print(parent,indivLocName)

In [60]:
fo.close()